<a href="https://colab.research.google.com/github/jay-kanakia/GenAI/blob/main/04_Pytorch_training_pipeline_using_nn_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)
df.sample(3)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
195,B,12.91,16.33,82.53,516.4,0.07941,0.05366,0.03873,0.02377,0.1829,...,13.88,22.00,90.81,600.6,0.1097,0.1506,0.17640,0.08235,0.3024,0.06949
385,M,14.60,23.29,93.97,664.7,0.08682,0.06636,0.08390,0.05271,0.1627,...,15.79,31.71,102.20,758.2,0.1312,0.1581,0.26750,0.13590,0.2477,0.06836
436,B,12.87,19.54,82.67,509.2,0.09136,0.07883,0.01797,0.02090,0.1861,...,14.45,24.38,95.14,626.9,0.1214,0.1652,0.07127,0.06384,0.3313,0.07735


In [5]:
X=df.iloc[:,1:]
y=df.iloc[:,0]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

ss=StandardScaler()
X_train_scaled=ss.fit_transform(X_train)
X_test_scaled=ss.transform(X_test)

le=LabelEncoder()
y_train_transformed=le.fit_transform(y_train)
y_test_transformed=le.transform(y_test)

In [7]:
X_train_tensor=torch.from_numpy(X_train_scaled.astype(np.float32))
X_test_tensor=torch.from_numpy(X_test_scaled.astype(np.float32))
y_train_tensor=torch.from_numpy(y_train_transformed.astype(np.float32))
y_test_tensor=torch.from_numpy(y_test_transformed.astype(np.float32))

In [25]:
import torch.nn as nn
class MyNN(nn.Module):

  def __init__(self,num_features):

    super().__init__()
    self.linear=nn.Linear(num_features,1)
    self.sigmoid=nn.Sigmoid()

  def forward(self,features):
    out=self.linear(features)
    out=self.sigmoid(out)
    return out


In [26]:
model=MyNN(X_train_tensor.shape[1])
for i in  model.parameters():
  print(i)

Parameter containing:
tensor([[ 0.0152,  0.1151,  0.0112, -0.1399, -0.0933,  0.1151,  0.1794,  0.0346,
          0.0563, -0.1577, -0.0107, -0.1032,  0.1262,  0.1053,  0.0755, -0.1192,
          0.1205,  0.0828, -0.0517,  0.0841, -0.1250,  0.1437, -0.0510,  0.0580,
         -0.0801,  0.0511,  0.1793,  0.1537, -0.1358,  0.0141]],
       requires_grad=True)
Parameter containing:
tensor([-0.1191], requires_grad=True)


In [34]:
learning_rate=0.01
epochs=1000

model=MyNN(X_train_tensor.shape[1])

optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

loss_function=nn.BCELoss()

In [35]:
for epoch in range(epochs):

  y_pred=model(X_train_tensor)

  loss=loss_function(y_pred,y_train_tensor.unsqueeze(1))

  optimizer.zero_grad()

  loss.backward()

  optimizer.step()

  print(f"epoch is {epoch+1} loss is {loss.item()}")

epoch is 1 loss is 1.0614094734191895
epoch is 2 loss is 1.0249381065368652
epoch is 3 loss is 0.9900281429290771
epoch is 4 loss is 0.956666111946106
epoch is 5 loss is 0.9248323440551758
epoch is 6 loss is 0.8945016264915466
epoch is 7 loss is 0.8656426668167114
epoch is 8 loss is 0.8382192254066467
epoch is 9 loss is 0.8121898174285889
epoch is 10 loss is 0.7875086069107056
epoch is 11 loss is 0.7641261219978333
epoch is 12 loss is 0.741990327835083
epoch is 13 loss is 0.7210468053817749
epoch is 14 loss is 0.7012401819229126
epoch is 15 loss is 0.6825140118598938
epoch is 16 loss is 0.6648127436637878
epoch is 17 loss is 0.6480809450149536
epoch is 18 loss is 0.6322645545005798
epoch is 19 loss is 0.617310643196106
epoch is 20 loss is 0.6031680107116699
epoch is 21 loss is 0.5897873044013977
epoch is 22 loss is 0.5771211385726929
epoch is 23 loss is 0.5651241540908813
epoch is 24 loss is 0.5537532567977905
epoch is 25 loss is 0.5429675579071045
epoch is 26 loss is 0.532728314399719

In [36]:
with torch.no_grad():
  y_pred=model(X_test_tensor)
  y_pred=torch.round(y_pred)
  accuracy=(y_pred==y_test_tensor).float().mean()
  print(accuracy.item())

0.5301631093025208
